# Shor's Algorithm
Shor's algorithm basically comes down to period-finding. Say you have a periodic function:

$$ f(x) = a^x \bmod N$$

where $a, N$ are positive integers with $a<N$ and they have no common factors. The period $r$ is the smallest non-zero integer such that:

$$a^r \bmod N = 1 $$

## How do we get from a period $r$ to factorisation?
So say we want to factor the number $N$ into it's prime factors. The procedure is the following:

1. Choose a random number $a$ between 1 and $N-1$.

2. Check that $a$ isn't already a non-trivial factor of $N$.

3. Perform quantum phase estimation to find $r$.

4. Notice that 

    $$(a^r - 1) \bmod N = 0 $$

    i.e. we must have that $N$ must divide into $(a^r - 1)$.
    For $r$ being *even*, we have:

    $$a^r -1 = (a^{r/2}-1)(a^{r/2}+1)$$
    
    For $r$ being *odd*, try a different $a$.
    
5. Now compute the greatest common divisor of $N$ and either $a^{r/2}-1$, or $a^{r/2}+1$, and test to see if either of these is a non-trivial factor. If so, return the factor. Else the algorithm fails.

Note there is a probability of at least 1/2 that $r$ is even.

## How does quantum phase estimation help find $r$?

The quantum algorithm to find $r$ is just the phase estimation algorithm applied to the unitary operator

$$U|y\rangle \equiv |ay(\bmod N)\rangle$$.

From here, it's clear that an eigenstate of $U$ is:

$$\begin{aligned}
|u_s\rangle &= \tfrac{1}{\sqrt{r}}\sum_{k=0}^{r-1}{e^{-\tfrac{2\pi i s k}{r}}|a^k \bmod N\rangle}\\[10pt]
U|u_s\rangle &= e^{\tfrac{2\pi i s}{r}}|u_s\rangle 
\end{aligned}
$$

We can use the phase estimation algorithm to obtain the corresponding eigenvalues $e^{2\pi is/r}$, where 

$$0 \leq s \leq r-1.$$

We don't want to have to try to construct an individual eigenstate of $U$, since that involves knowing $r$ - i.e. not going to happen. So we have to be a bit clever. Conveniently,

$$ \tfrac{1}{\sqrt{r}}\sum_{s=0}^{r-1} |u_s\rangle = |1\rangle$$

so if we perform quantum phase estimation on $U$, using the state $|1\rangle$, we measure a phase: $$\phi = \frac{s}{r},$$ where $s$ is a random integer between 1 and $r-1$.

## What do we have left to do?

1. Now that we have the phase $\phi$, we need to find $r$. We can do this with a [continued fractions](https://en.wikipedia.org/wiki/Continued_fraction) algorithm (or if you're lazy, just by looking at the result & guessing).

2. What is the circuit $U$? And in doing the phase estimation algorithm, isn't the operation $U^{2j}$ exponentially scaling? Luckily to calculate $U^{2^j}|y\rangle = |a^{2^j}y \bmod N\rangle$, we can just square $a$, $j$ times.

    For the example below (i.e. factorising 15) I will give the required $U$ operation. But this is a bottleneck of Shor's algorithm.

## Some other quick notes
1. For an $L$-bit number $N$, you will need to begin with $t = 2L + 1 + log(2 + \frac{1}{\epsilon})$ qubits on the counting register, where $\epsilon$ is the phase estimation error.
2. The Hadamard gates require $O(L)$ gates, and the inverse fourier transform $O(L^2)$ gates. The most costly part of the procedure is the modular exponentiation (i.e. applying the matrix $U$). This scales as $O(L^3)$.

# Task
Perform factorisation for $N = 15$. This extract from: M. Nielsen and I. Chuang, "Quantum Computation and Quantum Information" may be helpful.

![image1](images/qiqc.png)

Below is a function which applies a controlled-$U$ in qiskit, with random number $a$ and repeated *power* times.

In [ ]:
def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,7,8,11,13]:
        raise ValueError("'a' must be 2,7,8,11 or 13")
    U = QuantumCircuit(4)        
    for iteration in range(power):
        if a in [2,13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [7,8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a == 11:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U